
Facial Recognition using PyTorch and OpenCV

https://ritik12.medium.com/facial-recognition-using-pytorch-and-opencv-467c4e41d1f


Machine Learning - Face Recognition CNN Pytorch.ipynb
https://github.com/rubencg195/Pytorch-Tutorials/blob/master/Machine%20Learning%20-%20Face%20Recognition%20CNN%20Pytorch.ipynb



Face Recognition Using Pytorch
https://github.com/timesler/facenet-pytorch

In [1]:
from torch.optim import lr_scheduler
from torch.nn.init import *
from torchvision import transforms, utils, datasets, models
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torchvision import datasets
from torchvision.transforms import ToTensor

from FaceFeatureDataset import FaceFeatureDataset

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


Using cuda device


In [5]:
batch_size = 10
epochs = 10

In [2]:
linear_layer_2d = nn.Linear(in_features=64, out_features=32)
flatten = nn.Flatten()
# 1st dimension (128) = batch dimension, input 64 x 2
input_2d = torch.randn(10, 2, 64)
# print(input_2d)
print(input_2d.shape)
output_2d = linear_layer_2d(input_2d)
print(output_2d.size())

output_1d = flatten(input_2d)
print(output_1d.size())


torch.Size([10, 2, 64])
torch.Size([10, 2, 32])
torch.Size([10, 128])


In [3]:

testload = pd.read_csv("./outimg/Train/facefeature.csv")
landmarks = np.array(testload.iloc[0, 1:])
landmarks = landmarks.astype('float').reshape(-1, 2)
print(landmarks)

dataiter = iter(testload)
landmark = next(dataiter)
print(landmark)
# print(landmarks.shape)


[[0.15214901 0.26409625]
 [0.15847385 0.35434336]
 [0.17404998 0.4446534 ]
 [0.19193892 0.53497918]
 [0.21678205 0.62303933]
 [0.262472   0.70661546]
 [0.32675891 0.77644054]
 [0.40500139 0.83479593]
 [0.48582416 0.85384914]
 [0.56685146 0.84283572]
 [0.6481305  0.79481721]
 [0.71564276 0.73051458]
 [0.7716696  0.65456918]
 [0.80464692 0.56690236]
 [0.82840444 0.47454697]
 [0.8521777  0.37987876]
 [0.86440259 0.28281907]
 [0.20106432 0.21354466]
 [0.24754095 0.18147987]
 [0.30772143 0.17495049]
 [0.36313466 0.189205  ]
 [0.41614068 0.21732068]
 [0.58269505 0.21382786]
 [0.64068857 0.18878019]
 [0.70321333 0.17764091]
 [0.76562795 0.18269135]
 [0.81392966 0.22233966]
 [0.50349283 0.29655437]
 [0.50303656 0.3636261 ]
 [0.50025174 0.43299491]
 [0.4951541  0.50234799]
 [0.43711337 0.5343341 ]
 [0.46480426 0.54377418]
 [0.49713651 0.5509329 ]
 [0.52726608 0.54188616]
 [0.55509857 0.53051087]
 [0.27217955 0.29960666]
 [0.30701342 0.2790273 ]
 [0.35326978 0.27934197]
 [0.39471171 0.30737899]


In [7]:
# Create DataLoader 

training_data = FaceFeatureDataset(feature_file="./outimg/Train/facefeature.csv", label_file="./Dataset/Train/csv/train.csv")
test_data = FaceFeatureDataset(feature_file="./outimg/Test/facefeature.csv", label_file="./Dataset/Test/csv/test.csv")

train_loader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# 데이터 로드 확인 
for X, y in train_loader:
    print(f"Shape of X [N, F, C]: {X.shape}") # N , Channel, H= width W = height
    # print(X[0][0])
    print(f"Shape of Tensor y: {y.shape} {y.dtype}")   
    break

n_total_steps = len(train_loader)
print(f'Traing dat length {n_total_steps}')

Shape of X [N, F, C]: torch.Size([10, 68, 2])
Shape of Tensor y: torch.Size([10, 33]) torch.float32
Traing dat length 10


In [17]:
dataiter = iter(train_loader)
landmark, labels = next(dataiter)

print("Shape ", landmark.shape)
linear1 = nn.Linear(68*2, 32)
flatten = nn.Flatten()
x = flatten(landmark)
print(x.shape)
x = linear1(x)

# print('linear1', x.shape)


Shape  torch.Size([10, 68, 2])
torch.Size([10, 136])


RuntimeError: mat1 and mat2 must have the same dtype

In [ ]:

num_classes = 33

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()

        self.linear_relu_stack = nn.Sequential(
            nn.Linear(68, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes),
            nn.ReLU(),
        )
    

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)


In [ ]:
# criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction error
        pred = model(X)
        print(pred)
        
        loss = loss_fn(pred, y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
# Training
epochs = 10

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, criterion, optimizer)    
print("Done!")

In [ ]:
PATH = './cifar_net.pth'
torch.save(model.state_dict(), PATH)

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0,0
    with torch.no_grad():
        for X,y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            print(pred)



In [ ]:
for t in range(epochs):
    test(test_loader, model, criterion)

print("Done !")